This file includes all the steps involved in the creation of the project.

#### Libraries

In [49]:
import re
import time
import numpy as np
import pandas as pd
import os
from datetime import datetime
import logging
from logging.handlers import RotatingFileHandler
from appium import webdriver
from appium.options.android import UiAutomator2Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

handler = RotatingFileHandler('app.log', maxBytes=5000, backupCount=2)
logging.basicConfig(
    handlers=[handler],
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.getLogger('selenium').setLevel(logging.WARNING)
logging.getLogger('urllib3').setLevel(logging.WARNING)
logging.getLogger('appium').setLevel(logging.WARNING)

#### Log Configuration

In [2]:
console_handler = logging.StreamHandler()
file_handler = logging.FileHandler('app.log')

logging.basicConfig(
    handlers=[console_handler, file_handler],
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

#### Initializing Driver

In [52]:
# Device information (keep these as environment variables)
DEVICE_NAME = os.getenv('DEVICE_NAME', 'R9ZWA07928P')
DEVICE_UDID = os.getenv('DEVICE_UDID', 'R9ZWA07928P')

desired_caps = {
    'platformName': 'Android',
    'deviceName': DEVICE_NAME,
    'udid': DEVICE_UDID,
    'platformVersion': '14',
    'appPackage': 'com.instagram.android',
    'appActivity': '.activity.MainTabActivity',
    'automationName': 'UiAutomator2',
    'noReset': True,
    'newCommandTimeout': 6000,  # Reduced timeout
    'adbExecTimeout': 20000,  # Increased timeout
    'autoGrantPermissions': True,
    'disableWindowAnimation': True,
    'unicodeKeyboard': True,
    'resetKeyboard': True,
    "appium:ensureWebviewsHavePages": True
}

try:
    # First, check if device is connected
    logging.info("Checking if device is connected...")
    import subprocess
    adb_devices = subprocess.check_output('adb devices').decode()
    # logging.info(adb_devices)
    
    if DEVICE_UDID not in adb_devices:
        # logging.error(f"Device {DEVICE_UDID} not found. Please connect your device and enable USB debugging.")
        raise Exception(f"Device {DEVICE_UDID} not found.")
    
    options = UiAutomator2Options().load_capabilities(caps=desired_caps)
    driver = webdriver.Remote('http://127.0.0.1:4723/wd/hub', options=options)
    # logging.info("Driver initialized successfully")
    
    # Add a small delay to ensure the app is fully loaded
    import time
    time.sleep(5)
    
except Exception as e:
    # logging.error(f"Error initializing driver: {str(e)}")
    raise

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 74, in emit
    self.doRollover()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 179, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 115, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\hp\\OneDrive\\Documents\\Code Playground\\Shivam Projects\\instagram_project\\app.log' -> 'c:\\Users\\hp\\OneDrive\\Documents\\Code Playground\\Shivam Projects\\instagram_project\\app.log.1'
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen

#### X-PATHS

In [42]:
x_paths = {
    'search_explore': '//android.widget.FrameLayout[@content-desc="Search and explore"]',
    'search_bar': '//android.widget.EditText[@resource-id="com.instagram.android:id/action_bar_search_edit_text"]',
    'search_bar_text': '//android.widget.EditText[@resource-id="com.instagram.android:id/action_bar_search_edit_text"]',
    'message_button': '//android.widget.Button[@content-desc="Message"]',
    'message_text': '//android.widget.EditText[@resource-id="com.instagram.android:id/row_thread_composer_edittext"]',
    'send_button': '//android.widget.ImageView[@content-desc="Send"]'
}

#### Steps

In [20]:
def find_element(x_path_name):
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, x_paths[x_path_name])))
        return element
    except Exception as e:
        print(e)
        raise

In [ ]:
user_names = ['navneet_ks_', 'candidpavscene', 'sevainternal']
user_id = []
message_sent = []
date = []

search_explore = find_element('search_explore')
search_explore.click()
time.sleep(1)
search_bar = find_element('search_bar')
search_bar.click()
time.sleep(1)
count = 0
for username in user_names: 
    print(f"Searching for: {username}")
    search_bar_text = find_element('search_bar_text')
    search_bar_text.click()
    search_bar_text.clear()  # clear previous input
    search_bar_text.send_keys(username)
    user_id.append(username)
    search_result = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//android.widget.TextView[@resource-id="com.instagram.android:id/row_search_user_username"]')))
    search_result.click()
    time.sleep(3)
    message_button = find_element('message_button')
    message_button.click()
    time.sleep(2)
    message_text = find_element('message_text')
    message_text.click()
    message = f'test: {count}'
    message_text.send_keys(message)
    message_sent.append(message)
    time.sleep(2)
    send_button = find_element('send_button')
    send_button.click()
    date.append(datetime.now())
    time.sleep(2)
    # After clicking on the user, go back to search
    driver.back()
    driver.back()
    count += 1



In [56]:
output = pd.DataFrame({
    'user_id':user_id,
    'message_sent':message_sent,
    'date':date
})

In [57]:
output

,user_id,message_sent,date
0,navneet_ks_,test: 0,2025-03-24 14:52:41.442486
1,candidpavscene,test: 1,2025-03-24 14:53:03.666794
2,sevainternal,test: 2,2025-03-24 14:53:24.778122
